# Using deep features to train an image classifier

In [1]:
import turicreate

# Load some data

In [2]:
image_train = turicreate.SFrame('image_train_data/')
image_test = turicreate.SFrame('image_test_data/')

# Explore this image data

In [6]:
turicreate.visualization.set_target('auto')
image_train['image'].explore()

ValueError: Value must be a nonnegative integer or None

# Train an image classifier on raw image pixels

In [8]:
raw_pixel_model = turicreate.logistic_classifier.create(image_train,
                                                       target = 'label',
                                                       features = ['image_array'])

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.



Logistic regression:

--------------------------------------------------------

Number of examples          : 1904

Number of classes           : 4

...

| 2         | 11       | 2.891888  | 0.090885     | 0.391807          | 0.336634            |

| 3         | 13       | 1.014622  | 0.114806     | 0.443803          | 0.396040            |

| 4         | 14       | 1.014622  | 0.133832     | 0.453782          | 0.425743            |

| 9         | 21       | 1.000000  | 0.236034     | 0.533613          | 0.485149            |

+-----------+----------+-----------+--------------+-------------------+---------------------+

# Make predictions using simple raw pixel model

In [14]:
image_test[0:3]['image'].explore()

ValueError: Value must be a nonnegative integer or None

In [15]:
image_test[0:3]['label']

dtype: str
Rows: 3
['cat', 'automobile', 'cat']

In [16]:
raw_pixel_model.predict(image_test[0:3])

dtype: str
Rows: 3
['bird', 'cat', 'bird']

# Evaluate the raw pixel model on the test data

In [17]:
raw_pixel_model.evaluate(image_test)

{'accuracy': 0.483,
 'auc': 0.7321028333333335,
 'confusion_matrix': Columns:
 	target_label	str
 	predicted_label	str
 	count	int
 
 Rows: 16
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |     dog      |    automobile   |  104  |
 |     bird     |       bird      |  527  |
 |  automobile  |       bird      |  111  |
 |     bird     |    automobile   |  143  |
 |  automobile  |    automobile   |  641  |
 |     cat      |       dog       |  340  |
 |     dog      |       dog       |  455  |
 |     dog      |       bird      |  247  |
 |     cat      |    automobile   |  160  |
 |     cat      |       cat       |  309  |
 +--------------+-----------------+-------+
 [16 rows x 3 columns]
 Note: Only the head of the SFrame is printed.
 You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.,
 'f1_score': 0.4787751621552814,
 'log_loss': 1.1864477510040468,
 'precision'

# Train image classifier using deep features

In [12]:
len(image_train)

2005

In [18]:
deep_learning_model = turicreate.load_model('imagenet_model_iter45')
image_train['deep_features'] = deep_learning_model.extract_features(image_train)

OSError: Directory imagenet_model_iter45 does not exist

In [14]:
image_train

id,image,label,deep_features,image_array
24,Height: 32 Width: 32,bird,"[0.242871761322,1.09545373917, 0.0, ...","[73.0, 77.0, 58.0, 71.0,68.0, 50.0, 77.0, 69.0, ..."
33,Height: 32 Width: 32,cat,"[0.525087952614, 0.0,0.0, 0.0, 0.0, 0.0, ...","[7.0, 5.0, 8.0, 7.0, 5.0,8.0, 5.0, 4.0, 6.0, 7.0, ..."
36,Height: 32 Width: 32,cat,"[0.566015958786, 0.0,0.0, 0.0, 0.0, 0.0, ...","[169.0, 122.0, 65.0,131.0, 108.0, 75.0, ..."
70,Height: 32 Width: 32,dog,"[1.12979578972, 0.0, 0.0,0.778194487095, 0.0, ...","[154.0, 179.0, 152.0,159.0, 183.0, 157.0, ..."
90,Height: 32 Width: 32,bird,"[1.71786928177, 0.0, 0.0,0.0, 0.0, 0.0, ...","[216.0, 195.0, 180.0,201.0, 178.0, 160.0, ..."
97,Height: 32 Width: 32,automobile,"[1.57818555832, 0.0, 0.0,0.0, 0.0, 0.0, ...","[33.0, 44.0, 27.0, 29.0,44.0, 31.0, 32.0, 45.0, ..."
107,Height: 32 Width: 32,dog,"[0.0, 0.0,0.220677852631, 0.0, ...","[97.0, 51.0, 31.0, 104.0,58.0, 38.0, 107.0, 61.0, ..."
121,Height: 32 Width: 32,bird,"[0.0, 0.23753464222, 0.0,0.0, 0.0, 0.0, ...","[93.0, 96.0, 88.0, 102.0,106.0, 97.0, 117.0, ..."
136,Height: 32 Width: 32,automobile,"[0.0, 0.0, 0.0, 0.0, 0.0,0.0, 7.5737862587, 0.0, ...","[35.0, 59.0, 53.0, 36.0,56.0, 56.0, 42.0, 62.0, ..."
138,Height: 32 Width: 32,bird,"[0.658935725689, 0.0,0.0, 0.0, 0.0, 0.0, ...","[205.0, 193.0, 195.0,200.0, 187.0, 193.0, ..."


# Given the deep features, train a logistic classifier

In [19]:
deep_features_model = turicreate.logistic_classifier.create(image_train,
                                                           target='label',
                                                           features = ['deep_features'])

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.



Logistic regression:

--------------------------------------------------------

Number of examples          : 1904

Number of classes           : 4

...

| 2         | 8        | 0.156250  | 0.169759     | 0.771534          | 0.801980            |

| 3         | 13       | 0.312500  | 0.264500     | 0.802521          | 0.792079            |

| 4         | 14       | 0.390625  | 0.302775     | 0.824055          | 0.752475            |

| 9         | 19       | 1.000000  | 0.508055     | 0.913340          | 0.851485            |

+-----------+----------+-----------+--------------+-------------------+---------------------+

# Apply the deep features classifier on the first few images

In [20]:
image_test[0:3]['image'].explore()

ValueError: Value must be a nonnegative integer or None

In [21]:
deep_features_model.predict(image_test[0:3])

dtype: str
Rows: 3
['cat', 'automobile', 'cat']

# Quantitatively evaluate deep features classifier on test data

In [22]:
deep_features_model.evaluate(image_test)

{'accuracy': 0.7925,
 'auc': 0.9403266666666662,
 'confusion_matrix': Columns:
 	target_label	str
 	predicted_label	str
 	count	int
 
 Rows: 16
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |     cat      |       cat       |  665  |
 |     bird     |    automobile   |   12  |
 |  automobile  |    automobile   |  947  |
 |  automobile  |       dog       |   11  |
 |     cat      |       bird      |   77  |
 |     bird     |       dog       |   61  |
 |  automobile  |       bird      |   18  |
 |     bird     |       bird      |  807  |
 |  automobile  |       cat       |   24  |
 |     dog      |    automobile   |   6   |
 +--------------+-----------------+-------+
 [16 rows x 3 columns]
 Note: Only the head of the SFrame is printed.
 You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.,
 'f1_score': 0.793568246957149,
 'log_loss': 0.6381498130781853,
 'precision'